Device configuration

In [1]:
import torch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Hyper-parameters:

In [3]:
num_epochs = 80
batch_size = 100
learning_rate = 0.001

Image preprocessing

In [4]:
import torchvision.transforms as transforms

In [5]:
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(p=0.4),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])

CIFAR10 dataset:

In [22]:
from torch.utils.data import DataLoader
import torchvision

In [23]:
train_dataset = torchvision.datasets.CIFAR10(
    root=r'D:\pytorch_data', train=True, download=True, transform=transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root=r'D:\pytorch_data', train=False, download=True, transform=transform
)

100.0%


Extracting D:\pytorch_data\cifar-10-python.tar.gz to D:\pytorch_data
Files already downloaded and verified


In [24]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Build model:

In [25]:
import torch.nn as nn

In [ ]:
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels=in_channels, 
                     out_channels=out_channels, kernel_size=3, 
                     stride=1, padding='same', bias=False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)    
        self.conv2 = conv3x3(out_channels, out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        return self.relu(out)
    
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes):
        super().__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])